### K-means analysis_ver3 : 외부온도, 내부온도 category
- xlsx : clustering_inouttemp_k_3

In [1]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
csv_path = "/home/miserver10/workspace_vscode/datasets/data/*.csv"

csv_files = glob.glob(csv_path)

df_list = [pd.read_csv(csv_file) for csv_file in csv_files]
df = pd.concat(df_list, ignore_index=True)  #ignore_index = 기존 인덱스 무시하여 새로운 인덱스 생성(연속으로 이어진다)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339932 entries, 0 to 339931
Data columns (total 54 columns):
 #   Column                                   Non-Null Count   Dtype  
---  ------                                   --------------   -----  
 0   Front_Seat_LH_Temp Unit: Degree ('c)     339932 non-null  float64
 1   Front_Seat_RH_Temp Unit: Degree('c)      339932 non-null  float64
 2   Rear_Seat_CT_Temp Unit: Degree('c)       339932 non-null  float64
 3   Front_Roof_CT_Temp Unit: Degree('c)      339932 non-null  float64
 4   Front_Roof_LH_Temp Unit: Degree('c)      339932 non-null  float64
 5   Front_Roof_RH_Temp Unit: Degree('c)      339932 non-null  float64
 6   Rear_Roof_LH_Temp Unit: Degree('c)       339932 non-null  float64
 7   Rear_Roof_RH_Temp Unit: Degree('c)       339932 non-null  float64
 8   DRV_LH_Vent_Wind_Velocity Unit: m / s    339932 non-null  float64
 9   DRV_RH_Vent_Wind_Velocity Unit: m / s    339932 non-null  float64
 10  DRV_Foot_Vent_Wind_Velocity Unit

In [3]:
drop_list = ["Front_Seat_LH_Temp Unit: Degree ('c)",
       "Front_Seat_RH_Temp Unit: Degree('c)",
       "Rear_Seat_CT_Temp Unit: Degree('c)",
       "Front_Roof_CT_Temp Unit: Degree('c)",
       "Front_Roof_LH_Temp Unit: Degree('c)",
       "Front_Roof_RH_Temp Unit: Degree('c)",
       "Rear_Roof_LH_Temp Unit: Degree('c)",
       "Rear_Roof_RH_Temp Unit: Degree('c)",
       "DRV_LH_Vent_Wind_Velocity Unit: m / s",
       "DRV_RH_Vent_Wind_Velocity Unit: m / s",
       "DRV_Foot_Vent_Wind_Velocity Unit: m / s",
       "DRV_Seat_Vent_Wind_Velocity Unit: m / s",
       "PAS_LH_Vent_Wind_Velocity Unit: m / s",
       "PAS_RH_Vent_Wind_Velocity Unit: m / s",
       "PAS_Foot_Vent_Wind_Velocity Unit: m / s",
       "PAS_Seat_Vent_Wind_Velocity Unit: m / s", "5V_ADC_CH1 Unit: V",
       "5V_ADC_CH2 Unit: V", "5V_ADC_CH3 Unit: V", "5V_ADC_CH4 Unit: V",
       "5V_ADC_CH5 Unit: V", "PWM_CH8 Unit: PWM", "Frequency_CH8 Unit: Hz",
       "CR_Datc_DrModeAct","CR_Datc_TempAct", "CR_Datc_IntakeAct", "CR_Datc_AdsAct",
       "CR_Datc_DrVentTemp_C", "CR_Datc_DrFloorTemp_C", 
       "CR_Datc_ClimatePowSum_W", "LAT_ACCEL_m/s^2", "LONG_ACCEL_m/s^2",
       "YAW_RATE_'/s", "WHL_SPD_FL_km/h", "WHL_SPD_FR_km/h", "WHL_SPD_RL_km/h",
       "WHL_SPD_RR_km/h", 'CR_Datc_AcnCompSpdFB_rpm', 'CR_Datc_AptVal_PSI', 'CR_Datc_OutTemp_C', 
       'CR_Datc_OutTempC' ,'CR_Datc_EvapSnsTemp_C', 'CR_Datc_DrTempDispC', 'DATC_ModeDisp']

df = df.drop(drop_list, axis=1)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339932 entries, 0 to 339931
Data columns (total 10 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   CR_Datc_AcnCompPwrCon_W  339932 non-null  int64  
 1   CR_Datc_OutTempSnr_C     339932 non-null  float64
 2   CR_Datc_PTCPwrCon_W      339932 non-null  int64  
 3   CF_Datc_Heatpump_Mode    339932 non-null  int64  
 4   CR_Datc_SetTemp_C        339932 non-null  float64
 5   CR_Datc_IncarSnsTemp_C   339932 non-null  float64
 6   DATC_BlowerLevel         339932 non-null  int64  
 7   CR_Datc_PhotoSnsFB_V     339932 non-null  float64
 8   CR_Datc_Humidity         339932 non-null  float64
 9   CR_Mcu_VehSpd_Kph_sum    339932 non-null  float64
dtypes: float64(6), int64(4)
memory usage: 25.9 MB


In [5]:
df.rename(columns={'CR_Datc_OutTempSnr_C': '외부 온도', 'CR_Datc_AcnCompPwrCon_W': '에어컨 압축기 전력 소비량',
                   'CR_Datc_PTCPwrCon_W': '히터의 전력 소비량', 'DATC_BlowerLevel': '바람세기', 'CF_Datc_Heatpump_Mode' : '히트 펌프',
                   'CR_Datc_SetTemp_C': '설정 온도', 'CR_Datc_IncarSnsTemp_C': '내부 온도', 
                   'CR_Datc_PhotoSnsFB_V' : '일사량', 'CR_Datc_Humidity' : '습도', 'CR_Mcu_VehSpd_Kph_sum' : '차량 속도'}, inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339932 entries, 0 to 339931
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   에어컨 압축기 전력 소비량  339932 non-null  int64  
 1   외부 온도           339932 non-null  float64
 2   히터의 전력 소비량      339932 non-null  int64  
 3   히트 펌프           339932 non-null  int64  
 4   설정 온도           339932 non-null  float64
 5   내부 온도           339932 non-null  float64
 6   바람세기            339932 non-null  int64  
 7   일사량             339932 non-null  float64
 8   습도              339932 non-null  float64
 9   차량 속도           339932 non-null  float64
dtypes: float64(6), int64(4)
memory usage: 25.9 MB


In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
에어컨 압축기 전력 소비량,339932.0,253.707359,299.143430,0.0,0.000000,200.000000,420.000000,2010.000000
외부 온도,339932.0,18.767186,10.624092,-9.0,11.000000,20.500000,26.500000,39.000000
히터의 전력 소비량,339932.0,375.223221,734.062488,0.0,0.000000,0.000000,230.000000,5890.000000
히트 펌프,339932.0,1.443548,2.550282,0.0,0.000000,0.000000,0.000000,7.000000
설정 온도,339932.0,21.260543,1.945197,0.0,20.000000,21.000000,22.500000,27.000000
내부 온도,339932.0,23.957999,2.936478,0.0,22.500000,24.000000,26.000000,35.500000
바람세기,339932.0,3.372416,1.334061,0.0,3.000000,4.000000,4.000000,9.000000
일사량,339932.0,0.293877,0.228326,0.0,0.100000,0.200000,0.440000,1.240000
습도,339932.0,43.334913,15.393059,0.0,32.000000,42.500000,53.500000,90.000000
차량 속도,339932.0,54.541180,37.157516,0.0,18.138672,63.289062,85.609375,169.335938


In [8]:
#bins_temp = [-9, -5, -1, 3, 7, 11, 15, 19, 23, 27, 31, 35, 39]  # 외부 온도의 범위 설정
#labels_temp = ['-9-(-5)도', '-5-(-1)도', '31-34도']  # 각 범위의 레이블 설정

# 외부 온도를 4도씩 묶기 위한 범위 설정
bins_outtemp = list(range(-9, 40, 4))  # -9도에서 39도까지 4도 간격으로
labels_outtemp = [f'{bins_outtemp[i]}-{bins_outtemp[i+1]}' for i in range(len(bins_outtemp)-1)]  # 레이블 설정

In [9]:
df['외부온도범위'] = pd.cut(df['외부 온도'], bins=bins_outtemp, labels=labels_outtemp, right=False)

# pd.cut의 파라미터 right = True(기본값)일 경우, include_lowest=True 필요 : 왼쪽 값이 열린 구간이 되면서 첫 번째 경계 값이 포함되지 않게 되는것을 막아준다 
# right = Flase 일때는 오른쪽 경계값이 열린 구간이 되어 문제 해결

In [10]:
# 내부 온도를 6도씩 묶기 위한 범위 설정
bins_intemp = list(range(0, 37, 6))  # 0도에서 36도까지 6도 간격으로
labels_intemp = [f'{bins_intemp[i]}-{bins_intemp[i+1]}' for i in range(len(bins_intemp)-1)]  # 레이블 설정

In [11]:
df['내부온도범위'] = pd.cut(df['내부 온도'], bins=bins_intemp, labels=labels_intemp, right=False)

# pd.cut의 파라미터 right = True(기본값)일 경우, include_lowest=True 필요 : 왼쪽 값이 열린 구간이 되면서 첫 번째 경계 값이 포함되지 않게 되는것을 막아준다 
# right = Flase 일때는 오른쪽 경계값이 열린 구간이 되어 문제 해결

In [12]:
df.head()

,에어컨 압축기 전력 소비량,외부 온도,히터의 전력 소비량,히트 펌프,설정 온도,내부 온도,바람세기,일사량,습도,차량 속도,외부온도범위,내부온도범위
0,420,22.5,0,0,18.0,27.5,4,0.48,30.0,0.0,19-23,24-30
1,420,22.5,0,0,18.0,27.5,4,0.48,30.0,0.0,19-23,24-30
2,420,22.5,0,0,18.0,27.5,4,0.48,30.0,0.0,19-23,24-30
3,420,22.5,0,0,18.0,27.5,4,0.46,30.0,0.0,19-23,24-30
4,420,22.5,0,0,18.0,27.5,4,0.46,30.0,0.0,19-23,24-30


In [13]:
category_counts = df['외부온도범위'].value_counts().sort_index()

category_counts

외부온도범위
-9--5     7890
-5--1     4503
-1-3     10488
3-7      36215
7-11     25452
11-15    28124
15-19    36754
19-23    48978
23-27    57994
27-31    33672
31-35    39004
35-39    10857
Name: count, dtype: int64

In [14]:
category_counts = df['내부온도범위'].value_counts().sort_index()

category_counts

내부온도범위
0-6           2
6-12        252
12-18     10614
18-24    138970
24-30    186449
30-36      3645
Name: count, dtype: int64

In [15]:
cross_tab = pd.crosstab(df['외부온도범위'], df['내부온도범위'])

cross_tab

내부온도범위,0-6,6-12,12-18,18-24,24-30,30-36
외부온도범위,,,,,,
-9--5,0,58,624,4446,2762,0
-5--1,0,0,2,2276,2222,3
-1-3,2,47,777,3442,6220,0
3-7,0,147,3262,15614,16962,230
7-11,0,0,1367,14142,9932,11
11-15,0,0,4064,14696,8710,654
15-19,0,0,518,20650,15528,58
19-23,0,0,0,21336,27642,0
23-27,0,0,0,31287,26707,0


In [16]:
excel_filename = 'cross_tab_results_2.xlsx'
cross_tab.to_excel(excel_filename)

print(f"교차 테이블 결과가 '{excel_filename}' 파일로 저장되었습니다.")

교차 테이블 결과가 'cross_tab_results_2.xlsx' 파일로 저장되었습니다.


In [18]:
# 카테고리별 데이터 저장을 위한 딕셔너리 생성
category_data = {}

# 외부 온도 범위와 차량 속도 범위에 따라 데이터를 나누고 저장
for outtemp_range in df['외부온도범위'].unique():
    for intemp_range in df['내부온도범위'].unique():
        subset = df[(df['외부온도범위'] == outtemp_range) & (df['내부온도범위'] == intemp_range)]
        subset = subset.drop(['외부온도범위', '내부온도범위'], axis=1)  # '외부온도범위', '내부온도범위' 열 제외
        if len(subset) >= 30 :  # 데이터가 30개 이상인지 확인
            category_data[f'out_{outtemp_range}_in_{intemp_range}'] = subset

# 딕셔너리에 저장된 데이터프레임 출력
for key, value in category_data.items():
    print(f"카테고리: {key}")
    print(value)
    print()  

카테고리: out_19-23_in_24-30
        에어컨 압축기 전력 소비량  외부 온도  히터의 전력 소비량  히트 펌프  설정 온도  내부 온도  바람세기   일사량  \
0                  420   22.5           0      0   18.0   27.5     4  0.48   
1                  420   22.5           0      0   18.0   27.5     4  0.48   
2                  420   22.5           0      0   18.0   27.5     4  0.48   
3                  420   22.5           0      0   18.0   27.5     4  0.46   
4                  420   22.5           0      0   18.0   27.5     4  0.46   
...                ...    ...         ...    ...    ...    ...   ...   ...   
320647             210   20.0           0      0   23.0   25.5     4  0.06   
320648             210   20.0           0      0   23.0   25.5     4  0.06   
320649             210   20.0           0      0   23.0   25.5     4  0.06   
320650             210   20.0           0      0   23.0   25.5     4  0.06   
320651             210   20.0           0      0   23.0   25.5     4  0.06   

          습도     차량 속도  
0       30.0 

In [19]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# 스케일러 정의
scaler = StandardScaler()

# 결과를 저장할 엑셀 파일 이름
excel_file = 'clustering_imouttemp_k_3.xlsx'

# 엑셀 파일 생성
with pd.ExcelWriter(excel_file) as writer:
    for key, value in category_data.items():
        # 선택한 특성에 대해 스케일링 수행
        scaled_value = value.copy()  # 원본 데이터 복사
        scaled_value = scaler.fit_transform(scaled_value)

        # KMeans 적용
        kmeans = KMeans(n_clusters=3, random_state=42)
        clusters = kmeans.fit_predict(scaled_value)

        # 클러스터링 결과를 데이터프레임에 추가
        clustered_value = value.copy()
        clustered_value['클러스터'] = clusters

        # 클러스터별 평균 계산 (스케일링 되기 전의 값으로)
        cluster_means = clustered_value.groupby('클러스터').mean()

        # 결과를 반올림 (소수점 둘째 자리)
        cluster_means = cluster_means.round(1)

        # 결과를 엑셀 파일의 시트로 저장
        cluster_means.to_excel(writer, sheet_name=key)

# 엑셀 파일 저장 완료 메시지 출력
print(f"클러스터링 결과가 '{excel_file}' 파일에 저장되었습니다.")

/home/miserver10/workspace_vscode/K-means/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/miserver10/workspace_vscode/K-means/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/miserver10/workspace_vscode/K-means/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/miserver10/workspace_vscode/K-means/lib/python3.8/site-packages/sklearn/cluster/_kmeans.

클러스터링 결과가 'clustering_imouttemp_k_3.xlsx' 파일에 저장되었습니다.


/home/miserver10/workspace_vscode/K-means/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/miserver10/workspace_vscode/K-means/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
